In [ ]:
import time
from collections import defaultdict

import requests

In [ ]:
stop_words = requests.get(
    "https://gist.githubusercontent.com/sebleier/554280/raw/7e0e4a1ce04c2bb7bd41089c9821dbcf6d0c786c/NLTK's%2520list%2520of%2520english%2520stopwords"
).text.splitlines()
len(stop_words)

In [ ]:
bad_words_data = requests.get(
    "https://raw.githubusercontent.com/citiususc/VERY-NEG-and-VERY-POS-Lexicons/master/VERY-NEG%20B%3D1.csv"
).text.splitlines()
bad_words = []
for line in bad_words_data:
    line = line.split(",")
    if line[2] == "MN":
        if line[0] not in bad_words:
            bad_words.append(line[0])
len(bad_words)
bad_words[:15]

In [ ]:
good_words_data = requests.get(
    "https://raw.githubusercontent.com/citiususc/VERY-NEG-and-VERY-POS-Lexicons/master/VERY-POS%20B%3D2.csv"
).text.splitlines()
good_words = []
for line in good_words_data:
    line = line.split(",")
    if line[2] == "MP":
        if line[0] not in good_words:
            good_words.append(line[0])
len(good_words)
good_words[:15]

In [ ]:
class RedditClient:
    def __init__(self):
        self.headers = {"User-Agent": "User-Bot 0.1 by Some User"}
        self.client = requests.Session()
        self.client.headers = self.headers

    def getUrlJson(self, url):
        return self.client.get(url).json()

In [ ]:
username = ""

In [ ]:
r = RedditClient()

In [ ]:
comments = []
after = None

url = f"https://api.reddit.com/user/{username}/?limit=10"
print(url)
data = r.getUrlJson(url)
after = data["data"]["after"]
while len(comments) < 10000:
    url = f"https://api.reddit.com/user/{username}/?limit=10&after=" + after
    print(url, len(comments))
    data = r.getUrlJson(url)
    after = data["data"]["after"]
    if after is None:
        break
    comments.extend(data["data"]["children"])
    time.sleep(0.5)

In [ ]:
len(comments)

In [ ]:
subreddits = defaultdict(int)
for post in comments:
    subreddits[post["data"]["subreddit_name_prefixed"]] += 1
subreddits = dict(subreddits)
dict(sorted(subreddits.items(), key=lambda item: item[1], reverse=True))

In [ ]:
print("Top posts")
for post in sorted(comments, key=lambda a: a["data"]["ups"], reverse=True)[:10]:
    if post["kind"] == "t1":
        print(post["data"]["ups"])
        print(post["data"]["subreddit"])
        print(post["data"]["body"])
        print()

In [ ]:
print("Bottom posts")
for post in sorted(comments, key=lambda a: a["data"]["ups"])[:10]:
    if post["kind"] == "t1":
        print(post["data"]["ups"])
        print(post["data"]["subreddit"])
        print(post["data"]["body"])
        print()

In [ ]:
words = defaultdict(int)
for post in comments:
    if post["kind"] == "t1":
        for word in post["data"]["body"].strip().replace("'", "").lower().split():
            if word not in stop_words:
                words[word] += 1

word = dict(words)
scored_words = dict(sorted(words.items(), key=lambda item: item[1], reverse=True))

In [ ]:
good_score = 0
for word in scored_words:
    if word in good_words:
        good_score += 1
good_score

In [ ]:
bad_score = 0
for word in scored_words:
    if word in bad_words:
        bad_score -= 1
bad_score

In [ ]:
good_score + bad_score

In [ ]:
len(scored_words)

In [ ]:
import time
from collections import defaultdict

In [ ]:
class RedditClient:
    def __init__(self):
        self.headers = {"User-Agent": "User-Bot 0.1 by Some User"}
        self.client = requests.Session()
        self.client.headers = self.headers

    def getUrlJson(self, url):
        return self.client.get(url).json()

In [ ]:
def getComments(username):
    r = RedditClient()
    comments = []
    after = None

    url = f"https://api.reddit.com/user/{username}/?limit=100"
    print(url)
    data = r.getUrlJson(url)
    comments.extend(data["data"]["children"])
    after = data["data"]["after"]
    while len(comments) < 10000:
        url = f"https://api.reddit.com/user/{username}/?limit=100&after=" + after
        print(url, len(comments), "+", len(data["data"]["children"]))
        data = r.getUrlJson(url)
        after = data["data"]["after"]
        comments.extend(
            filter(
                None,
                [
                    item if item["kind"] == "t1" else None
                    for item in data["data"]["children"]
                ],
            )
        )
        if after is None:
            break
        time.sleep(0.5)
    return comments

In [ ]:
import requests


def doInvestigation(reddit_user):
    log = []
    comments = getComments(reddit_user)
    subreddits = defaultdict(int)
    for post in comments:
        subreddits[post["data"]["subreddit_name_prefixed"]] += 1
    subreddits = dict(subreddits)
    dict(sorted(subreddits.items(), key=lambda item: item[1], reverse=True))
    print("Top posts")

    for post in sorted(comments, key=lambda a: a["data"]["ups"], reverse=True)[:10]:
        print(post["data"]["ups"])
        print(post["data"]["subreddit"])
        print(post["data"]["body"])
        print(post["data"]["link_permalink"])
        print()
    print()
    print("Bottom posts")
    for post in sorted(comments, key=lambda a: a["data"]["ups"])[:10]:
        print(post["data"]["ups"])
        print(post["data"]["subreddit"])
        print(post["data"]["body"])
        print(post["data"]["link_permalink"])
        print()

In [ ]:
def hasEmojiReaction(message, emoji):
    return emoji in [i["emoji"] for i in message["reactions"]]

In [ ]:
doInvestigation("")